In [1]:
import pandas as pd
import glob
import os
import geopandas as gpd
import re
from shapely import wkt

/home/xuyuan/.local/lib/python3.8/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_28810/2891674836.py:4: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly

## classification of real estate brokerages

We first classify the real estate brokerages of the whole country into several main parts by the first four names, and then we extract 200 main real estate brokerages in the whole country and extract the first two words in the result. Then after extracting, we give label to each of the dataset and remerging the dataset to calculate the HHI index and so on.

## cleaning the dataset

In [16]:
# read all the csv files in the directory
for i in range(16, 23):
    folder_path = '/home/xuyuan/Desktop/RealEstateBrokerage/classifying brokerages/whole dataset/whole/{}/'.format(i)
    csv_files = glob.glob(folder_path + '/**/*.csv', recursive=True)

    dfs = pd.DataFrame()
    for file in csv_files:
        df = pd.read_csv(os.path.join(folder_path, file))
        # print(file)
        dfs = pd.concat([dfs, df], ignore_index=True)
    dfs.to_csv('/home/xuyuan/Desktop/RealEstateBrokerage/classifying brokerages/combined_result/{}_whole.csv'.format(i), index=False)

In [22]:
# read all the csv files in the directory
for i in range(16, 23):
    folder_path = '/home/xuyuan/Desktop/RealEstateBrokerage/classifying brokerages/whole dataset/lianjia_beke/20{} cleaned/'.format(i)
    csv_files = glob.glob(folder_path + '/**/*.csv', recursive=True)
    
    dfs = pd.DataFrame()
    for file in csv_files:
        df = pd.read_csv(os.path.join(folder_path, file))
        # print(file)
        dfs = pd.concat([dfs, df], ignore_index=True)
    dfs.to_csv('/home/xuyuan/Desktop/RealEstateBrokerage/classifying brokerages/combined_result/{}_lianjia.csv'.format(i), index=False)

## extracting the first two words

## now I need to merge our dataset with the labels

In [2]:
def extract_chinese(text):
    return ''.join(re.findall(r'[\u4e00-\u9fff]+', text))

def extract_number(text):
    return ''.join(re.findall(r'[0-9]+', text))

In [64]:
for i in range(16, 23):
    folder_path = '/home/xuyuan/Desktop/RealEstateBrokerage/classifying brokerages/broker/'
    csv_files = glob.glob(folder_path + '/**/*.csv', recursive=True)

    dfs = pd.DataFrame()
    for file in csv_files:
         
        df = pd.read_csv(os.path.join(folder_path, file))
        df['year'] = extract_number(file)
        df['Chinese_Name'] = extract_chinese(file)
        df = df[~df['name'].str.contains('职业介绍|黄马褂|临时工|停车场|便利店|口腔诊所|自助银行|鲜花|数码冲印')]
        df.loc[df['name'].str.contains(r'链家|贝壳|Lianjia|lianjia|壳点|自如', na=False), 'name'] = '链家'
        # print(file)
        dfs = pd.concat([dfs, df], ignore_index=True)

In [65]:
print(len(dfs))
# 506353
# 505672

505672


In [66]:
dfs['year'].unique()

array(['16', '22', '21', '20', '19', '18', '17'], dtype=object)

In [71]:
dfs.to_csv('/home/xuyuan/Desktop/RealEstateBrokerage/classifying brokerages/combined_result/16_22_broker_10cities.csv', index=False)

In [72]:
data = pd.read_csv('/home/xuyuan/Desktop/RealEstateBrokerage/classifying brokerages/combined_result/16_22_broker_10cities.csv')
df = pd.read_csv('/home/xuyuan/Desktop/RealEstateBrokerage/classifying brokerages/combined_result/check.csv')
data['first_two'] = data['name'].str[0:2]

In [73]:
df['label'] = df.index

In [74]:
final_df = pd.merge(data, df, left_on='first_two', right_on='two', how='left')

In [75]:
final_df.to_csv('/home/xuyuan/Desktop/RealEstateBrokerage/classifying brokerages/combined_result/16_22_broker_10cities_label.csv', index=False)

In [76]:
data = pd.read_csv('/home/xuyuan/Desktop/RealEstateBrokerage/classifying brokerages/combined_result/16_22_broker_10cities_label.csv')

In [77]:
unique_chinese_names = data['Chinese_Name'].unique()
unique_years = data['year'].unique()

In [78]:
unique_years

array([16, 22, 21, 20, 19, 18, 17])

In [79]:
for chinese_name in unique_chinese_names:
    for year in unique_years:
        # Create a subset based on the current Chinese name and year
        subset = data[(data['Chinese_Name'] == chinese_name) & (data['year'] == year)]

        file_name = f'{year}_{chinese_name}.csv'
        subset.to_csv(os.path.join('/home/xuyuan/Desktop/RealEstateBrokerage/classifying brokerages/cleaned', file_name), index=False)

## now final clean the result

In [5]:
data = pd.read_csv('../cleaned_district_Jan_4.csv')
dataframes_list = []
for i in data['chinese_csv'].unique():
    dataframes_list.append(data[data['chinese_csv'] == i])

for i in range(len(dataframes_list)):
    dataframes_list[i] = gpd.GeoDataFrame(dataframes_list[i], geometry=dataframes_list[i]['geometry'].apply(wkt.loads))
    dataframes_list[i] = dataframes_list[i].set_crs('epsg:4326')

In [6]:
unique_chinese_names = data['chinese_csv'].unique()
chinese_name_to_index = {name: i for i, name in enumerate(unique_chinese_names)}


In [7]:
chinese_name_to_index['北京市17.csv']

1

In [8]:
folder_path = '/home/xuyuan/Desktop/RealEstateBrokerage/classifying brokerages/cleaned'
csv_files = glob.glob(folder_path + '/**/*.csv', recursive=True)

In [9]:
valid_csv_files = []

for file in csv_files:
    city_name = extract_chinese(file)
    number = extract_number(file)
    file_name = '{}{}.csv'.format(city_name, number)

    # Check if file_name is in the dictionary
    if file_name in chinese_name_to_index:
        valid_csv_files.append(file)  # Add to the valid list if the condition is met

csv_files = valid_csv_files

for file in csv_files:
    city_name = extract_chinese(file)
    number = extract_number(file)
    file_name = '{}{}.csv'.format(city_name, number)

In [10]:
import find_stores_within

for file in csv_files:
    city_name = extract_chinese(file)
    number = extract_number(file)
    file_name = '{}{}.csv'.format(city_name, number)
    i = chinese_name_to_index[file_name]
        
    df = pd.read_csv(os.path.join(folder_path, file))

    store_locations = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df['gpsx'], df['gpsy']))    
    store_locations['label'].fillna(-1, inplace = True)
    store_locations_cpp = [list(point.coords[0]) for point in store_locations['geometry']]
        
    communities_gdf = dataframes_list[i]
    community_locations_cpp = [list(point.coords[0]) for point in communities_gdf['geometry']]
    
    
    labels = [int(label) for label in store_locations['label'].tolist()]
    threshold_distance = 0.42 # 420 meters
    result = find_stores_within.find_stores_within(store_locations_cpp, 
                                               community_locations_cpp, 
                                               labels,
                                               threshold_distance)
    
    dataframes_list[i]['label'] = result[0]
    
    # we first calculate for each row, how many lianjia are within the distance of the community.
    dataframes_list[i]['lianjia_420'] = dataframes_list[i]['label'].apply(lambda x: x.count(0))

## now we use the label to construct the HHI index

In [11]:
dataframes_list[0]['label']

0                  [1, 4, 0, 0]
1                   [0, -1, -1]
2       [0, 0, 1, 27, -1, 1, 1]
3                [28, 4, 0, 28]
4                           [0]
                 ...           
5434                         []
5435             [0, 28, 1, 28]
5436                       [-1]
5437                    [1, -1]
5438                [0, -1, -1]
Name: label, Length: 5439, dtype: object

g++ -O3 -shared -std=gnu++17 -fPIC -I/usr/include/python3.8 -I/home/xuyuan/.local/lib/python3.8/site-packages/pybind11/include -o hhi_calculator.so hhi_calculator.cpp

In [12]:
import hhi_calculator 

for i in range(0, len(dataframes_list)):
    data = dataframes_list[i]['label'].tolist()
    
    # you also need to clculate the number of elements in the list
    dataframes_list[i]['broker_420'] = [len(sublist) for sublist in data]
    hhi_indices = hhi_calculator.calculate_HHI(data)
    dataframes_list[i]['hhi'] = hhi_indices

In [18]:
final_data = pd.DataFrame()
for i in range(0, len(dataframes_list)):
    dataframes_list[i]['hhi'].fillna(0, inplace=True)
    final_data = pd.concat([final_data, dataframes_list[i]], ignore_index=True)

In [19]:
print(len(final_data))

222837


In [21]:
final_data.to_csv('../cleaned_district_Jan_5.csv', index=False)